In [1]:
import torch
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from torchvision import transforms
import numpy as np

import sys
sys.path.append("C:/Users/ozhyb/licenseplate-segmentation")
from model import create_model

%load_ext autoreload
%autoreload 2

model_path = 'C:/Users/ozhyb/licenseplate-segmentation/model.pth'
video_path = 'C:/Users/ozhyb/Desktop/Test2.mp4'
output_video_path = 'C:/Users/ozhyb/Desktop/video_tracked23.mp4'
threshold = 0.1

model = create_model(outputchannels=1, aux_loss=False)

checkpoint = torch.load(model_path, map_location='cpu')
state_dict = checkpoint['model']

filtered_state_dict = {k: v for k, v in state_dict.items() if 'aux_classifier' not in k}

model.load_state_dict(filtered_state_dict, strict=False)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def pred(image, model, device):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(image).to(device)
    input_batch = input_tensor.unsqueeze(0)

    with torch.no_grad():
        output = model(input_batch)['out'][0]
        return output

# Читання відео
video = cv2.VideoCapture(video_path)
fps = int(video.get(cv2.CAP_PROP_FPS))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
dim = (frame_width, frame_height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_tracked = cv2.VideoWriter(output_video_path, fourcc, fps, dim)

while True:
    ret, frame = video.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)

    output = pred(pil_image, model, device)

    output = output.squeeze().cpu().numpy()
    output = cv2.resize(output, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

    mask = output > threshold
    frame_rgb[mask] = [255, 0, 0]

    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    video_tracked.write(frame_bgr)

video.release()
video_tracked.release()

print("Обробка відео завершена. Збережено як", output_video_path)


Обробка відео завершена. Збережено як C:/Users/ozhyb/Desktop/video_tracked23.mp4
